In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms
import torch.nn.functional as F
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test
from sklearn.metrics import mean_absolute_error
import os
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path

In [2]:
def onoff_error(pred, gt, threshold):
    abs_error = np.abs(pred-gt)
    error = [x for x in abs_error.reshape(1, -1).tolist()[0] if x >= threshold]
    
    return np.mean(error)

In [76]:
dataset = 5
gid = 0
srv = 1
c = 0
cmd = {}
rnn_tree_valid_error = {}
num_iterations = 3000
lr = 0.01
p = 0.0
bidirectional = True
hidden_size = 50
for fold_num in range(5):
    rnn_tree_valid_error[fold_num] = {}
    for cell_type in ['GRU', 'LSTM', 'RNN']:
        rnn_tree_valid_error[fold_num][cell_type] = {}
        for num_layers in [1, 2]:
            rnn_tree_valid_error[fold_num][cell_type][num_layers] = {}
            for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw', 'residual'])):
    #             print(fold_num, lr, order)
    #             if order[0] == 'hvac':
    #                 continue
                o = "\', \'".join(str(x) for x in order)
                rnn_tree_valid_error[fold_num][cell_type][num_layers][order] = {}
                directory = "../code/baseline/rnn-tree/"
                folder = "{}/{}/{}/{}/{}/{}/{}/{}/{}/".format(dataset, fold_num, cell_type, hidden_size, 
                                                              num_layers, bidirectional, lr, num_iterations, p)
                filename = "test-error-{}-{}-{}-{}-{}-{}-{}-{}-{}-[\'{}\'].npy".format(fold_num, dataset, cell_type, 
                                                                                        hidden_size, num_layers, 
                                                                                        bidirectional, lr, num_iterations,
                                                                                        p, o)
                                                                                        
                full_path = directory + folder + filename
                my_file = Path(full_path)
                if not my_file.exists():
                    print(full_path)
                    o = " ".join(str(x) for x in order)
                    line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 3 {} {} 0 {} {} &".format(gid, lr, num_iterations, fold_num, o))
    #                 print(line)

                    c += 1
                    if c%1 == 0:
                        gid+=1
                    if gid == 4:
                        gid = 0
                        srv += 1
                else:
                    rnn_tree_valid_error[fold_num][cell_type][num_layers][order] = np.load(full_path).item()

print(c)                               

0


In [54]:
tensor = np.load("../2015-5appliances.numpy.npy")
dr = tensor[:, 3]

# calculate gt
test_gt = {}
valid_gt = {}
for fold_num in range(5):
    test_gt[fold_num] = {}
    valid_gt[fold_num] = {}
    train, test = get_train_test(1, 5, fold_num)
    valid = train[int(0.8*len(train)):].copy()
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        test_gt[fold_num][appliance] = test[:, idx+1]
        valid_gt[fold_num][appliance] = valid[:, idx+1]
        
threshold = {}
for appliance in ['dr', 'dw', 'mw']:
    sample_list = []
    for fold_num in range(5):
        sample_list = np.append(sample_list, [x for x in test_gt[fold_num][appliance].reshape(1, -1).tolist()[0] if x > 5])
    mean = np.mean(sample_list)
    print(appliance, mean)
    threshold[appliance] = 0.1*mean

dr 725.849570636
dw 285.780713239
mw 31.7082756317


In [77]:
rnn_tree_best_param = {}
for fold_num in range(5):
    rnn_tree_best_param[fold_num] = {}
    min_error = np.inf
    for cell_type in ['GRU', 'LSTM', 'RNN']:
        for num_layers in [1, 2]:
            for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw', 'residual'])):
                for it in range(1000, 3001, 1000):
                    error = 0
                    try:
                        for idx, appliance in enumerate(order):
                            if appliance == 'residual':
                                continue
                            error += rnn_tree_valid_error[fold_num][cell_type][num_layers][order][it][appliance]
#                             if appliance in ['hvac', 'fridge']:
#                                 error += mean_absolute_error(rnn_tree_valid_error[fold_num][cell_type][num_layers][order][it][appliance].reshape(-1, 24),
#                                                             valid_gt[fold_num][appliance].reshape(-1, 24))
#                             else:
#                                 error += onoff_error(rnn_tree_valid_error[fold_num][cell_type][num_layers][order][it][appliance].reshape(-1, 24),
#                                                             valid_gt[fold_num][appliance].reshape(-1, 24), threshold[appliance])
#                             print(error)
                        if error < min_error:
                            min_error = error
                            rnn_tree_best_param[fold_num]['cell_type'] = cell_type
                            rnn_tree_best_param[fold_num]['order'] = order
                            rnn_tree_best_param[fold_num]['iters'] = it
                            rnn_tree_best_param[fold_num]['num_layers'] = num_layers
                    except:
                        continue

                    

In [78]:
best_rnn_tree = rnn_tree_best_param
best_rnn_tree

{0: {'cell_type': 'LSTM',
  'iters': 3000,
  'num_layers': 2,
  'order': ('mw', 'hvac', 'fridge', 'residual', 'dw', 'dr')},
 1: {'cell_type': 'RNN',
  'iters': 3000,
  'num_layers': 1,
  'order': ('fridge', 'residual', 'dr', 'dw', 'hvac', 'mw')},
 2: {'cell_type': 'LSTM',
  'iters': 3000,
  'num_layers': 2,
  'order': ('dw', 'dr', 'fridge', 'residual', 'hvac', 'mw')},
 3: {'cell_type': 'LSTM',
  'iters': 3000,
  'num_layers': 2,
  'order': ('mw', 'dw', 'fridge', 'residual', 'hvac', 'dr')},
 4: {'cell_type': 'LSTM',
  'iters': 3000,
  'num_layers': 2,
  'order': ('mw', 'fridge', 'residual', 'dr', 'dw', 'hvac')}}

In [79]:
e = {}
for fold_num in range(5):
    cell_type = rnn_tree_best_param[fold_num]['cell_type']
    order = rnn_tree_best_param[fold_num]['order']
    it = rnn_tree_best_param[fold_num]['iters']
    num_layers = rnn_tree_best_param[fold_num]['num_layers']
    o = "\', \'".join(str(x) for x in order)
    e[fold_num] = pd.Series(rnn_tree_valid_error[fold_num][cell_type][num_layers][order][it])

In [80]:
pd.DataFrame(e).mean(axis=1)

dr           67.157928
dw           15.433289
fridge       30.019916
hvac        337.689863
mw            8.431509
residual    490.327572
dtype: float64

In [81]:
# get prediction for cnn tree
rnn_tree_pred = {}
for fold_num in range(5):
    rnn_tree_pred[fold_num] = {}
        
    
    cell_type = rnn_tree_best_param[fold_num]['cell_type']
    order = rnn_tree_best_param[fold_num]['order']
    it = rnn_tree_best_param[fold_num]['iters']
    num_layers = rnn_tree_best_param[fold_num]['num_layers']

    o = "\', \'".join(str(x) for x in order)
    
    directory = "../code/baseline/rnn-tree/"
    folder = "{}/{}/{}/{}/{}/{}/{}/{}/{}/".format(dataset, fold_num, cell_type, hidden_size, 
                                                  num_layers, bidirectional, lr, 3000, p)
    filename = "test-pred-{}-{}-{}-{}-{}-{}-{}-{}-{}-[\'{}\'].npy".format(fold_num, dataset, cell_type, 
                                                                                        hidden_size, num_layers, 
                                                                                        bidirectional, lr, 3000,
                                                                                        p, o)
                      

    full_path = directory + folder + filename
    rnn_pred = np.asarray(np.load(full_path).item()[it])
    
    for idx, appliance in enumerate(order):
        rnn_tree_pred[fold_num][appliance] = rnn_pred[idx].reshape(-1, 1, 112,24)
    

In [40]:
def calculate_error(pred, gt, threshold):
    error = {}
    overall = {}
    num_homes = {}
    
    # calculte number of homes in each fold
    for fold_num in range(5):
        num_homes[fold_num] = gt[fold_num]['hvac'].reshape(-1, 1, 112,24).shape[0]
    homes = pd.Series(num_homes).sum()
    
    # for Constantly On appliances: hvac and fridge
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        overall[appliance] = 0
        error[appliance] = {}
        for fold_num in range(5):
            error[appliance][fold_num] = mean_absolute_error(pred[fold_num][appliance].reshape(-1, 24), 
                                                             gt[fold_num][appliance].reshape(-1, 24))
            overall[appliance] += error[appliance][fold_num]*num_homes[fold_num]
        overall[appliance] /= homes
        
    # for ON/OFF appliances: dryer, dishwasher and microwave
#     for appliance in ['dr', 'dw', 'mw']:
#         error[appliance] = {}
#         overall[appliance] = 0                                                                
#         for fold_num in range(5):
#             error[appliance][fold_num] = onoff_error(pred[fold_num][appliance].reshape(-1, 24), 
#                                                      gt[fold_num][appliance].reshape(-1, 24), threshold[appliance])
#             overall[appliance] += error[appliance][fold_num]*num_homes[fold_num]
#         overall[appliance] /= homes
    
    
    return error, overall

In [82]:
pd.Series(calculate_error(rnn_tree_pred, test_gt, threshold)[1])

dr         66.950190
dw         15.444522
fridge     29.997301
hvac      337.350723
mw          8.413974
dtype: float64

In [42]:
active_threshold = {'hvac':230, 'fridge':20, 'dr':250, 'dw':55, 'mw':10}
num_homes = {0:14, 1:14, 2:14, 3:13, 4:13}

In [43]:
active = {}
for dataset in [1, 3]:
    active[dataset] = {}
    for fold_num in range(5):
        active[dataset][fold_num] = {}
        for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
            active[dataset][fold_num][appliance] = (test_gt[fold_num][appliance] > active_threshold[appliance])*1

In [44]:
# calculate gt
test_gt = {}
valid_gt = {}
for fold_num in range(5):
    test_gt[fold_num] = {}
    valid_gt[fold_num] = {}
    train, test = get_train_test(1, 5, fold_num)
    valid = train[int(0.8*len(train)):].copy()
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        test_gt[fold_num][appliance] = test[:, idx+1]
        valid_gt[fold_num][appliance] = valid[:, idx+1]

In [83]:
error = {}
for dataset in [3]:
    error[dataset] = {}
    for method in ['sc']:
        error[dataset][method] = {}
        error[dataset][method]['Active Error'] = {}
        error[dataset][method]['Inactive Error'] = {} 
        print("Dataset ", dataset)
        for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
            active_error = []
            inactive_error = []
            print(appliance)
            for fold_num in range(5):
                abs_error = np.abs(rnn_tree_pred[fold_num][appliance].reshape(-1, 112, 24)-test_gt[fold_num][appliance])
                active_error.append(abs_error[active[dataset][fold_num][appliance] == 1])
                inactive_error.append(abs_error[active[dataset][fold_num][appliance] == 0])
    #             print(appliance, fold_num, abs_error[active[dataset][fold_num][appliance] == 1].mean())
            print("active error", np.hstack(active_error).mean())
            print("inactive error", np.hstack(inactive_error).mean())



Dataset  3
hvac
active error 389.827301951
inactive error 188.689091963
fridge
active error 29.0661087485
inactive error 71.0407359471
dr
active error 1303.39629653
inactive error 2.39254857712
dw
active error 369.526442217
inactive error 0.414199764314
mw
active error 56.9265551837
inactive error 2.20991661404
